<a href="https://colab.research.google.com/github/SivanandhamMani/My-Project/blob/main/DeepSeek_Fine_Tuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments, Trainer
from datasets import load_dataset, Dataset
import torch

In [3]:
data = [
    {"text":"Patient: I have a headache.\nDoctor: Please take rest and stay hydrated"},
    {"text":"Patient: My stomach is upset.\nDoctor: Try a light diet and monitor your symptoms"},
    {"text":"Patient: I feel dizzy.\nDoctor: Make sure you sit down and drink some water"}
]

In [4]:
dataset = Dataset.from_list(data)
model_name = "deepseek-ai/deepseek-coder-1.3b-instruct"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

In [5]:
def tokenize_function(examples):
  tokens = tokenizer(examples['text'],
                     truncation=True,
                     max_length=64,
                     padding='max_length')
  tokens['labels'] = tokens['input_ids'].copy()
  return tokens


In [6]:
tokenized_dataset = dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/3 [00:00<?, ? examples/s]

In [7]:
training_args = TrainingArguments(
    output_dir='./fast_finetuned',
    num_train_epochs=2,
    per_device_train_batch_size=1,
    max_steps=10,
    logging_steps=1,
    no_cuda=True,
    fp16=False)

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1604: FutureWarning: using `no_cuda` is deprecated and will be removed in version 5.0 of 🤗 Transformers. Use `use_cpu` instead
  warnings.warn(


In [8]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset
)

In [ ]:
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Currently logged in as: sivanandhaman88 (svnm_ml) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


In [ ]:
trainer.save_model('./fast_finetuned')
tokenizer.save_pretrained('./fast_finetuned')

In [ ]:
def generate_response(prompt, max_length=64):
  input = tokenizer(prompt, return_tensors="pt")
  output = model.generate(**input, max_length=max_length)
  return tokenizer.decode(output[0], skip_special_tokens=True)

In [ ]:
prompt = "Patients: I have a headache"
print(("chatbot response:" generate_response(prompt)))